# COVID screening lab simulation

## Requirements and module imports

Code in this simulation uses a standard Anaconda Python environment (https://www.anaconda.com/distribution/#download-section). Additionally this model uses SimPy3 (https://simpy.readthedocs.io/en/latest/). Install SimPy3 with `pip install 'simpy<4'`.

In [1]:
import simpy
import inspect
from sim_utils.replication import Replicator
from sim_utils.parameters_200422 import Scenario

## Set up scenarios

Parameters defined in scenarios will overwrite default values in the parameters python file.

In [2]:
# Set up a dictionary to hold scenarios
scenarios = {}

scenarios['30k_recommended'] = Scenario(
    samples_per_day = 30000,

    resource_numbers = {
        'human_sample_receipt_shift_1': 15,
        'human_sample_receipt_shift_2': 9,
        'human_sample_prep_shift_1': 5,
        'human_sample_prep_shift_2': 5,
        'human_rna_prep_shift_1': 4,
        'human_rna_prep_shift_2': 4,
        'human_pcr_shift_1': 3,
        'human_pcr_shift_2': 3,
        'beckman_rna_extraction': 16,
        'pcr_plate_stamper': 2,
        'pcr_plate_reader': 13,
        'sample_prep_automation': 5
        },
    
    # Workstations represent the physical capacity  
    workstation_capacity = {
        'workstation_0': 99999,
        'workstation_1a': 7,
        'workstation_1b_man': 5,
        'workstation_1b_auto': 5, 
        'workstation_2': 16,
        'workstation_3': 2,
        'workstation_4': 13
        },

    # Resource available hours (use hours)
    resource_shift_hours = {
        'human_sample_receipt_shift_1': (0.3, 9.0),
        'human_sample_receipt_shift_2': (9.0, 17.75),
        'human_sample_prep_shift_1': (0.3, 9.0),
        'human_sample_prep_shift_2': (9.0, 17.75),
        'human_rna_prep_shift_1': (0.3, 9.0),
        'human_rna_prep_shift_2': (9.0, 17.75),
        'human_pcr_shift_1': (0.3, 9.0),
        'human_pcr_shift_2': (9.0, 17.75),
        'beckman_rna_extraction': (0.0, 24.0),
        'pcr_plate_stamper': (0.0, 24.0),
        'pcr_plate_reader': (0.0, 24.0),
        'sample_prep_automation': (0.0, 24.0),
        },
    
    # Range of times new jobs may start
    process_start_hours = {
        'sample_receipt': (0, 16),
        'sample_prep': (0, 16),
        'rna_extraction': (0, 16),
        'pcr_prep': (0, 24),
        'pcr': (0, 24)
        },

     # Resource unavailability on any whole day due to breakdown
    resource_breakdown_unavailability = {
        'human_sample_receipt_shift_1': 0,
        'human_sample_receipt_shift_2': 0,
        'human_sample_prep_shift_1': 0,
        'human_sample_prep_shift_2': 0,
        'human_rna_prep_shift_1': 0,
        'human_rna_prep_shift_2': 0,
        'human_pcr_shift_1': 0,
        'human_pcr_shift_2': 0,
        'beckman_rna_extraction': 0.0,
        'pcr_plate_stamper': 0.0,
        'pcr_plate_reader': 0.0,
        'sample_prep_automation': 0
        },
    
    # Process duration. Tuple of fixed time, time per entity, and time per item in entity.
    # Multi-step automated processes have three sets of times (set up, automated, clean down)
    process_duration = {
         'batch_input': ([0,0,0],),
         'sample_receipt': ([16, 0, 0],),
         'sample_prep_manual': ([37, 0, 0],),
         'sample_prep_auto': ([2, 0, 0], [6, 0, 0], [2, 0, 0]),
         'rna_extraction': ([5,0,0],[70,0,0],[2,0,0]),
         'pcr_prep': ([5,0,0],[10,0,0],[1,0,0]),
         'pcr': ([5,0,0],[117,0,0],[1,0,0]),
         },
             
    allow_maual_sample_prep = False,
    tea_break_times = [2*60, 6*60, 12*60, 16*60],
    meal_break_times = [4*60, 14*60],
    break_start_spread = 60,
    tea_break_duration = [15, 20],
    meal_break_duration = [30, 40],
    
    # rna pcr kanban group limit
    # Limit of PCR read capapcity multiple allowed from sample prep onwards
    pcr_kanban_limit = 3
    )

## Run model

In [3]:
replications = 30
replications = Replicator(scenarios, replications)
replications.run_scenarios()

                                                                               
Output results
--------------
                                                min          mean    median  \
                                             Result        Result    Result   
result_type               name                                                
demand_met                30k_recommended      0.99      0.990000      0.99   
input                     30k_recommended  29992.00  29992.000000  29992.00   
max_process_time_hours    30k_recommended     17.87     18.019667     17.98   
median_process_time_hours 30k_recommended     17.87     18.019667     17.98   
output                    30k_recommended  29808.00  29808.000000  29808.00   

                                                max  
                                             Result  
result_type               name                       
demand_met                30k_recommended      0.99  
input                     30k_recommended

## Show model default parameters

Run the code below to model defaults these are over-ridden by scenarios).

In [4]:
print(inspect.getsource(Scenario.__init__))

    def __init__(self, *initial_data, **kwargs):
        
        # Set default values
        # 16/4/2020 Adjust paramters so that day starts with FTE arrival


        # Work arrival
        self.samples_per_day = 30000
        self.deliveries_per_day = 1

        # Day and run parameters
        # 16/4/2020 Model is designed to run primarily in single days
        self.day_duration = 1440
        self.run_days = 1
        self.warm_up_days = 0
        
        # Breaks for people (high prority job, but does not interupt work)
        # Times from start of FTE day (6am)
        self.tea_break_times = [2*60, 6*60, 12*60, 16*60]
        self.meal_break_times = [4*60, 14*60]
        # Spread start of break for people randomly after set start times
        self.break_start_spread = 60
                
        # break duration is a uniform distribution between min and max
        self.tea_break_duration = [15, 20]
        self.meal_break_duration = [30, 40]
        
        # Audit parame